# 指文字(数字)認識 — 実験2
pivot_table を穴埋めし、SVM で分類してみよう。

---

### 本日のゴール
1. **pivot_table** の穴埋め → `X(180,42)` と `y(180,)` を生成  
2. **SVM** で学習・評価し、**正解率の変動** を観察  
3. **発展課題**: 座標シフト正規化で精度向上 & 好きな改善アイデアに挑戦  

> 提出物  
> - ノートブック (`exp2_<学籍番号>.ipynb`)  
> - accuracy 結果 (C, kernel 変更含む 3 回以上)  
> - 考察欄の記述  


## 0. ライブラリ読み込み

In [2]:
import numpy as np, pandas as pd, glob, os, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
print('✔️ libs ready')

ModuleNotFoundError: No module named 'numpy'

## 1. CSV を `data/` にアップロード
1. 左のフォルダ ▶ で `右クリック` → **`data`** フォルダ作成  
2. `1-01.csv`〜`9-20.csv` を **ドラッグ&ドロップ**  
3. 下セルで 180 枚あるか確認

In [ ]:
csv_files = sorted(glob.glob('data/*.csv'))
print('files:', len(csv_files))
assert len(csv_files)==180, '180 枚ない場合はアップロード漏れを確認'

## 2. `total` DataFrame を作成

In [ ]:
df_list=[]
for f in csv_files:
    label, num = os.path.splitext(os.path.basename(f))[0].split('-')
    tmp = pd.read_csv(f)
    tmp['label'] = int(label); tmp['num'] = int(num)
    df_list.append(tmp)
total = pd.concat(df_list, ignore_index=True)
df = total   # 別名 df (発展課題と合わせるため)
print('total shape:', total.shape)
total.head()

### 2‑1. ざっくり形状を散布図で見る

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
for lbl in range(1,10):
    tmp = total[total['label']==lbl]
    ax.scatter(tmp['x'], tmp['y'], s=10, alpha=0.6, label=str(lbl))
ax.set_aspect('equal')
ax.set_xlabel('x (0‑1)'); ax.set_ylabel('y (0‑1)')
ax.set_title('Landmarks by Label (color)')
ax.legend(title='digit')
plt.show()

## 3. pivot_table – 穴埋めチャレンジ

**目標:** `X.shape == (180, 42)` と `y.shape == (180,)` を作ること。

> **なぜ pivot?**  
> - 縦長 (3780 行) では “1 行 = 1 サンプル” になっていない  
> - pivot で **横長 (180 行 × 42 列)** に並び替えることで、  
>   機械学習ライブラリがそのまま扱える行列 **X** が手に入る

👇 のセルの `TODO` 部分（`A`〜`E`）に正しい式を代入し、下のテストセルで `✓ Passed` が出ればクリア。

【ヒント】`A`〜`E`には以下のいずれかが入る

index	x	y	z	label	num

3-0. ドキュメントを読んでみよう 🔍  
https://note.nkmk.me/python-pandas-pivot-table/ を読んで使い方を確認

In [ ]:
# ===== pivot_table を完成させよう =====
pivot = total.pivot_table(
    index=['A','B'],        # ① 180 行にしたい
    columns='C',            # ② 0〜20 の index を列にしたい
    values=['D','E']        # ③ x と y のみを使用したい
)
# 列 multi‑index → フラット名 (x0,y0,x1,y1,...)
pivot.columns = [f'{c}{i}' for c,i in pivot.columns]

# 行列・ラベルへ分割
X = pivot.values
y = pivot.index.get_level_values('label').values

print('X', X.shape, 'y', y.shape)

In [ ]:
# ---- テスト ----
assert X.shape==(180,42), 'X の shape が違います'
assert y.shape==(180,), 'y の shape が違います'
ref = total.query('label==1 & num==1').sort_values('index')
assert np.isclose(X[0,0], ref.iloc[0]['x']), 'x0 値が一致しません'
print('✓ Passed! pivot OK')

## 4. SVM 学習 & 初回評価

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)
clf = SVC(kernel='linear', C=10).fit(X_train, y_train)
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'1回目 accuracy={acc:.3f}')
ConfusionMatrixDisplay.from_predictions(y_test, pred, cmap='Blues'); plt.show()

## 5. 考察タスク (自動化セルあり)

### 5‑1. 繰り返し学習で精度のばらつきを見る

In [ ]:
def run_many(n=10, C=10, kernel='linear'):
    accs=[]
    for seed in range(n):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, stratify=y, random_state=seed)
        model = SVC(C=C, kernel=kernel).fit(X_train, y_train)
        accs.append(accuracy_score(y_test, model.predict(X_test)))
    return accs

acc_list = run_many()
print('10 runs acc:', acc_list)
print('mean=', np.mean(acc_list).round(3), 'std=', np.std(acc_list).round(3))

### 5‑2. `C` を変えて平均精度を比較

In [ ]:
for C in [0.1, 1, 10, 100]:
    accs = run_many(10, C=C)
    print(f'C={C:<4}  mean={np.mean(accs):.3f}')

### 5‑3. `kernel` を変えて比較

In [ ]:
for k in ['linear','rbf','poly','sigmoid']:
    accs = run_many(10, C=10, kernel=k)
    print(f'kernel={k:<8} mean={np.mean(accs):.3f}')

## 6. 発展課題1 — 最小値シフトで正規化

**目的:**  
各サンプル内で x, y の最小値を原点に移動し、  
*提示位置ズレ* の影響を減らして精度が上がるか試す。

👇 セルの `TODO` 箇所 (`A`, `B`) に正しい式を入れて実装してください。

In [ ]:
# SettingWithCopyWarning を無視
import warnings, copy
warnings.simplefilter('ignore')

max_label, max_num, max_index = 9, 20, 21
df_shifted = pd.DataFrame(columns=df.columns)

for l in range(max_label):
    for n in range(max_num):
        df_q = df.query(f'label=={l+1} & num=={n+1}').copy()
        # ===== TODO: 各サンプルの最小値を引く =====
        df_q.loc[:,'x'] = df_q['x'] - df_q['x'].min(axis=A)   # ← A
        df_q.loc[:,'y'] = df_q['y'] - df_q['y'].min(axis=A)   # ← A
        df_shifted = pd.concat([df_shifted, B])  # ← B

print('shifted shape', df_shifted.shape)

### 6‑1. シフト後散布図

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(df_shifted['x'], df_shifted['y'], c=df_shifted['index'], s=5, cmap='tab20')
plt.gca().set_aspect('equal'); plt.title('shifted landmarks'); plt.show()

### 6‑2. シフト後データで再学習

In [ ]:
pivot_s = df_shifted.pivot_table(index=['label','num'],
                                   columns='index', values=['x','y'])
pivot_s.columns = [f'{c}{i}' for c,i in pivot_s.columns]
Xs = pivot_s.values
ys = pivot_s.index.get_level_values('label').values

accs = run_many(10, C=10, kernel='linear')
accs_s = []
for seed in range(10):
    X_train, X_test, y_train, y_test = train_test_split(
        Xs, ys, test_size=0.3, stratify=ys, random_state=seed)
    accs_s.append(accuracy_score(
        SVC(C=10,kernel='linear').fit(X_train,y_train).predict(X_test), y_test))
print('original mean:', np.mean(accs).round(3),
      'shifted mean:', np.mean(accs_s).round(3))

## 7. 発展課題2（任意）
- (a) **さらに精度を上げる工夫**：学習データの追加・z軸も追加・モデル変更など  
- (b) **コード改善**：関数化・可視化ツール追加  
- (c) **識別対象を増やす**：0, 10 など  
- (d) **左右手の区別を無くす**：左右反転データ追加や handedness 用特徴を導入  
- (e) その他便利機能があれば自由に実装  

挑戦した内容と結果をノートに追記してください。

## 8. 最終考察 (まとめ)

_ここに最終まとめを書く_